In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np

In [3]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.thing = nn.Parameter(torch.rand(10), requires_grad=True)
    
#     def forward(self):
#         return self.thing

In [4]:
# net = Net()

# learning_rate = 0.1

# loss_fn = nn.MSELoss()

# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [5]:
# ideal_weights = torch.tensor([0.0027] * 10)
# for epoch in range(100):
#     optimizer.zero_grad()
#     pred = net()
#     print(pred)

#     weight = torch.zeros(10)
#     for i in range(10):
#         if pred[i].item() > 0.5:
#             weight[i] = 0.0027
#         else:
#             weight[i] = 0.00783
    
#     total_loss = ideal_weights - weight
#     print(ideal_weights)
#     print(weight)
#     print(total_loss)

#     loss = loss_fn(pred, total_loss.float())
#     print(loss)
#     loss.backward()
#     optimizer.step()

In [6]:
# class customAutograd(torch.autograd.Function):
#     @staticmethod
#     def forward(ctx, input):
#         """
#         In the forward pass we receive a Tensor containing the input and return
#         a Tensor containing the output. ctx is a context object that can be used
#         to stash information for backward computation. You can cache arbitrary
#         objects for use in the backward pass using the ctx.save_for_backward method.
#         """
#         ctx.save_for_backward(input)
#         return input / 2

#     @staticmethod
#     def backward(ctx, grad_output):
#         """
#         In the backward pass we receive a Tensor containing the gradient of the loss
#         with respect to the output, and we need to compute the gradient of the loss
#         with respect to the input.
#         """
#         input, = ctx.saved_tensors
#         return grad_output * input

In [7]:
# class testNet(nn.Module):
#     def __init__(self):
#         super(testNet, self).__init__()
#         torch.manual_seed(12)
#         self.elem_material = nn.Parameter(torch.rand(10), requires_grad=True)
    
#     def forward(self):
#         print(self.elem_material)

#         E_vector = 131 / (1 + torch.exp(-10 * (self.elem_material - torch.tensor([0.5] * 10)))) + 69
#         Stif = E_vector * 10
#         u = Stif / 2

#         return u


In [8]:
# net = testNet()

# num_epochs = 10
# learning_rate = 1
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

# ideal_u = torch.tensor([5.0] * 10)

# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     u = net()

#     loss = loss_fn(u, ideal_u)
#     loss.backward()

#     for name, param in net.named_parameters():
#         if param.requires_grad:
#             print(name, param.grad)

#     optimizer.step()

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
class testFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx, mat):
        ctx.save_for_backward(mat)

        return torch.linalg.inv(mat)
    
    def backward(ctx, grad_output):
        mat = ctx.saved_tensors[0]
        print(mat)
        matKron = torch.tensor(torch.kron(mat, mat), requires_grad=True)
        matKron.backward(torch.ones_like(matKron))
    
        return torch.mm(grad_output, matKron)

In [11]:
testFunction = testFunc.apply

In [12]:
class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.mat = nn.Parameter(torch.rand((20, 20), requires_grad=True, device=device))

    def forward(self):    
        return testFunction(self.mat)

In [13]:
# net = testNet()
# net.to(device)
# optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# loss_fn = nn.MSELoss()

# target = torch.zeros((20, 20), device=device)

# for epoch in range(10):
#     output = net()
#     loss = loss_fn(output, target)
#     loss.backward()
#     optimizer.step()

In [14]:
x = torch.rand((10, 10))
y = torch.linalg.inv(x)
z = torch.zeros((10, 10))
for i in range(10):
    for j in range(10):
        z[i][j] = y[i][j]
print(x.shape, x.stride(), '\n', x)
print(y.shape, y.stride(), '\n', y)
print(z.shape, z.stride(), '\n', z)

torch.Size([10, 10]) (10, 1) 
 tensor([[0.1918, 0.1803, 0.2155, 0.5611, 0.5720, 0.9786, 0.6170, 0.3775, 0.4423,
         0.5906],
        [0.6288, 0.0203, 0.6156, 0.9616, 0.5464, 0.6602, 0.5558, 0.8209, 0.6746,
         0.7378],
        [0.6018, 0.2573, 0.3906, 0.1765, 0.8904, 0.5973, 0.1033, 0.6991, 0.7451,
         0.1207],
        [0.5554, 0.1747, 0.7398, 0.0433, 0.8002, 0.5523, 0.8501, 0.4919, 0.0850,
         0.3248],
        [0.6316, 0.1739, 0.6015, 0.1284, 0.7329, 0.7275, 0.9069, 0.5207, 0.1282,
         0.3518],
        [0.9457, 0.8961, 0.2276, 0.1977, 0.5779, 0.2690, 0.7003, 0.3062, 0.7441,
         0.1640],
        [0.9599, 0.7538, 0.9345, 0.6052, 0.3923, 0.8109, 0.9056, 0.3868, 0.7032,
         0.8897],
        [0.5704, 0.9538, 0.6355, 0.8186, 0.2495, 0.7299, 0.5380, 0.8492, 0.8569,
         0.8880],
        [0.6319, 0.7336, 0.0688, 0.3943, 0.4307, 0.6660, 0.1119, 0.4437, 0.7585,
         0.6704],
        [0.3222, 0.4476, 0.0326, 0.9590, 0.5586, 0.8565, 0.2932, 0.9464, 0.074

In [15]:
mat1 = torch.tensor([1, 2, 3])
mat2 = torch.tensor([[3, 4, 5, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 3, 4, 5, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 3, 4, 5]])

mat1 @ mat2

tensor([ 3,  4,  5,  6,  8, 10,  9, 12, 15])

In [16]:
mat3 = torch.zeros((10, 100))
k = 0
for i in range(10):
    for j in range(10):
        mat3[i][j + k] = mat2[j]
    k = k + 10
mat3

RuntimeError: expand(torch.LongTensor{[9]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)

In [ ]:
def manualKron(mat1, mat2):
    initSize = 2
    kronSize = initSize ** 2
    kron = torch.zeros((kronSize, kronSize))
    x = 0
    for i in range(initSize):
        for k in range(initSize):
            y = 0
            for j in range(initSize):
                for l in range(initSize):
                    kron[x][y] = mat1[i][j].item() * mat2[k][l].item()
                    y += 1
            x += 1
    return kron

In [ ]:
mat1 = torch.tensor([[2.0, 2.0, 3.0, 0.0],
                     [2.0, 3.0, 4.0, 5.0],
                     [3.0, 4.0, 5.0, 6.0],
                     [0.0, 5.0, 6.0, 7.0]], requires_grad=True)
mat2 = torch.linalg.inv(mat1)
mat2

tensor([[ 0.0000, -0.2500,  0.5000, -0.2500],
        [-0.2500, -4.0625,  2.8750,  0.4375],
        [ 0.5000,  2.8750, -2.2500, -0.1250],
        [-0.2500,  0.4375, -0.1250, -0.0625]], grad_fn=<LinalgInvExBackward0>)

In [39]:
mat1 = torch.tensor([[1, 2]])
mat2 = torch.tensor([[3, 4], [5, 6]])
print(mat1)
print(mat2)
print(torch.transpose(mat1, 0, 1))
print(torch.transpose(mat2, 0, 1))

tensor([[1, 2]])
tensor([[3, 4],
        [5, 6]])
tensor([[1],
        [2]])
tensor([[3, 5],
        [4, 6]])


tensor([[ 3,  8],
        [ 5, 12]])

In [3]:
fullString = input("Enter a string: ")
num = input("Enter a number: ")

uniqueString = set(fullString)
uniqueString_list = []
for k in uniqueString:
    uniqueString_list.append(k)

repChar = []

for letter1 in uniqueString_list:

    count = 0

    for letter2 in fullString:
        if letter2 == letter1:
            count = count + 1

    if count == num:
        repChar.append(letter1)

print(repChar)


[]


In [4]:
import torch
mat1 = torch.tensor([[1, 2], [3, 5]]).float()
mat2 = torch.tensor([[1], [2]]).float()
torch.linalg.solve(mat1, mat2)

tensor([[-1.0000],
        [ 1.0000]])

In [4]:
# CS 177
# Lab: 07
# Filename: lab07.py
# Name: [Edith Azpeitia]
# Purdue Email: eazpeiti@purdue.edu
# Task description: Using Vigenere cipher 
# (input): A string and an integer key
# (output): After repeating given keyword it encrypts it

# Function 1
def generateKey(plain_text, key_text):
    # Making list of keytext/user input
    key_list = list(key_text)
    
    # Making if else statement
    # If keytext and plaintext are same normal keytext would be returned
    if len(plain_text) == len(key_text):
        return(key_text)

    # Else statement for when the keytext doesn't have enough characters for plaintext 
    else:
        for i in range(len(plain_text) - len(key_text)):
            key_list.append(key_text[i % len(key_text)])
            
    # Joining to make string
    return("" . join(key_list))


# Function 2
def encrypt(plain_text, generated_key):
    # Making an empty list
    cipher_text = []
    # Looping through each character in the length of PT
    for i in range(len(plain_text)):
        # Turning each character of PT and KT into its ASCII values then
        # adding them and finding modulus to find remainder
        v = (ord(plain_text[i]) + ord(generated_key[i])) % 128
        # Adding the new value of v back into list one by one
        cipher_text.append(chr(v))
        # Joining character in list to make a string
    return("" . join(cipher_text))
    
# Function 3
def decrypt(cipher_text, generated_key):
    x = []
    for i in range(len(cipher_text)):
        v = (ord(cipher_text[i]) - ord(generated_key[i]) + 128) % 128
        x.append(chr(v))
    return("" . join(x))

if __name__ == "__main__":
    
    plain_text = input("Please input the plaintext: ")
    key_text = input("Please input the key: ")
    genkey = generateKey(plain_text, key_text)
    encrypt = encrypt(plain_text, key_text)
    print("Generated Key: ", genkey)
    print(("Ciphertext: "), encrypt)
    print("Original/Decrypted Text:", decrypt(encrypt, genkey))


IndexError: string index out of range

In [8]:
import torch
mat1 = [[-1, 0, -2, 0],
        [0, -1, 0, -2],
        [-3, 0, -4, 0],
        [0, -3, 0, -4]]
tens1 = torch.tensor(mat1).float()
tens1trans = tens1.transpose(0, 1)
tens1[0]
tens1trans[0]

tensor([-1.,  0., -3.,  0.])

In [16]:
grad_output = torch.tensor([1, 2])
resid = torch.tensor([3, 4])
mat2 = [[1, 2], [3, 4]]
StifInv = torch.tensor(mat2).float()
StifInvTrans = torch.transpose(StifInv, 0, 1)

dStif = torch.zeros(4)

for i in range(2):
    for j in range(2):
        dStif[i * 2 + j] = grad_output[i] * resid[j]

print(dStif)
print(StifInv)
print(StifInvTrans)

final_grad = torch.zeros(4)
a = 0
b = 0
for i in range(4):
    kronCol = torch.kron(-StifInvTrans[a], StifInv[b])
    print(kronCol)
    b += 1
    if b == 2:
        a += 1
        b = 0
    final_grad[i] = dStif @ kronCol
print(final_grad)

tensor([3., 4., 6., 8.])
tensor([[1, 2],
        [3, 4]])
tensor([[1, 3],
        [2, 4]])
tensor([-1, -2, -3, -6])


RuntimeError: dot : expected both vectors to have same dtype, but found Float and Long